# Deep Agents

<img src="./media/deep_agent_diagram.png" width=400>

Language model based applications have moved from simple chat interfaces, to more 'agentic' frameworks. While the definition of what an 'agent' is varies, it generally refers to a system where an LLM is equipped with some tools, and is allowed to operate continuously. It is often with the intent to complete a more complex goal by chaining together multiple actions together, ultimately relying on the reasoning abilities of the language model to determine what steps to take and when to stop.

This initial approach has worked well, with agentic systems proving more useful, helpful, and capable than their naive chat completions beginnings, ushering in a new wave of interest and investment into arming LLMs with various integrations and tools to be able to perform more complex tasks. Although useful, there still remained a disconnect between the capability of an LLM with digitial tools and a human with the same tools, primarily around 'long-horizon' tasks.

<img src="./media/models-are-succeeding-at-increasingly-long-tasks.png" width=600>

[Measuring AI Ability to Complete Long Tasks](https://metr.org/blog/2025-03-19-measuring-ai-ability-to-complete-long-tasks/)

When observing a graph that compares success rates of models on human benchmarked time to complete tasks, most modern day models can complete tasks that often take 4 minutes for humans to complete successfully, however as task complexity grows success rate falls. Anecdotely, not many _useful_ tasks tend to fall into the 'takes four minutes to complete' category, i.e:

<img src="./media/easy_tasks.png" width=450>

[HCAST: Human-Calibrated Autonomy Software Tasks](https://metr.org/hcast.pdf)

With most of what would be considered 'productive' tasks often taking much more time and effort to complete. 

As LLMs become smarter and more capable through improved training techniques, their ability for successful long-horizon task completion has improved (reference prior graph), but even more gains have been made through custom frameworks meant to encourage and enable this kind of behavior. Popularized initially by workflows such as [Deep Research](https://blog.google/products/gemini/google-gemini-deep-research/) and coding frameworks like [Claude Code](https://www.anthropic.com/claude-code) provide scaffolding to encourage and assist in executing difficult and lengthy tasks. 

<img src="./media/da_la.png" width=400>

[Deep Agents](https://blog.langchain.com/deep-agents/)

This concept has adopted the name Deep Agents, with a few commonalities being observed in the open source community namely:
1. A detailed and specific **system prompt**
2. A **planning** or to-do list tool
3. A **file system** for context management
4. **Sub Agents!**

In this notebook, we'll discuss the theory behind why these techniques have proven successful for creating Deep Agents and ecouraging longer task execution, and provide an example of this all coming together by using open source frameworks to create our own Deep Agent!

---
## System Prompt

<img src="./media/system_prompt.png" width=300>

More robust and long running agents tend to have very detailed and beautifully written system prompts, some recent snapshots of popular agent prompts can be seen here:

- [Claude Code System Prompt](https://github.com/x1xhlol/system-prompts-and-models-of-ai-tools/blob/main/Claude%20Code/claude-code-system-prompt.txt)
- [Cursor Agent System Prompt](https://github.com/x1xhlol/system-prompts-and-models-of-ai-tools/blob/main/Cursor%20Prompts/Agent%20Prompt%202025-09-03.txt)
- [Manus System Prompt](https://github.com/x1xhlol/system-prompts-and-models-of-ai-tools/blob/main/Manus%20Agent%20Tools%20%26%20Prompt/Prompt.txt)

Notably, these prompts are roughly 200+ lines of code long (not counting tool instructions!), with specific instructions around tone, style, behavior, and tool use- often all blending together.

Claude Code provides pseudo examples, with placeholders indicating different actions and future behavior.

```
<example>
user: Help me write a new feature that allows users to track their usage metrics and export them to various formats

assistant: I'll help you implement a usage metrics tracking and export feature. Let me first use the TodoWrite tool to plan this task.
Adding the following todos to the todo list:
1. Research existing metrics tracking in the codebase
2. Design the metrics collection system
3. Implement core metrics tracking functionality
4. Create export functionality for different formats

Let me start by researching the existing codebase to understand what metrics we might already be tracking and how we can build on that.

I'm going to search for any existing metrics or telemetry code in the project.

I've found some existing telemetry code. Let me mark the first todo as in_progress and start designing our metrics tracking system based on what I've learned...

[Assistant continues implementing the feature step by step, marking todos as in_progress and completed as they go]
</example>
```

Manus includes styles and values

```
## Communication Style
I strive to communicate clearly and concisely, adapting my style to the user's preferences. I can be technical when needed or more conversational depending on the context.

## Values I Uphold
- Accuracy and reliability in information
- Respect for user privacy and data
- Ethical use of technology
- Transparency about my capabilities
- Continuous improvement
```

Cursor Agent has a whole coding style guide:

```
<code_style>
IMPORTANT: The code you write will be reviewed by humans; optimize for clarity and readability. Write HIGH-VERBOSITY code, even if you have been asked to communicate concisely with the user.

Naming
Avoid short variable/symbol names. Never use 1-2 character names
Functions should be verbs/verb-phrases, variables should be nouns/noun-phrases
Use meaningful variable names as described in Martin's "Clean Code":
Descriptive enough that comments are generally not needed
Prefer full words over abbreviations
Use variables to capture the meaning of complex conditions or operations
Examples (Bad → Good)
genYmdStr → generateDateString
n → numSuccessfulRequests
[key, value] of map → [userId, user] of userIdToUser
resMs → fetchUserDataResponseMs
Static Typed Languages
Explicitly annotate function signatures and exported/public APIs
Don't annotate trivially inferred variables
Avoid unsafe typecasts or types like any
Control Flow
Use guard clauses/early returns
Handle error and edge cases first
Avoid unnecessary try/catch blocks
NEVER catch errors without meaningful handling
Avoid deep nesting beyond 2-3 levels
Comments
Do not add comments for trivial or obvious code. Where needed, keep them concise
Add comments for complex or hard-to-understand code; explain "why" not "how"
Never use inline comments. Comment above code lines or use language-specific docstrings for functions
Avoid TODO comments. Implement instead
Formatting
Match existing code style and formatting
Prefer multi-line over one-liners/complex ternaries
Wrap long lines
Don't reformat unrelated code </code_style>
```

We can learn a lot from these prompting best practices of successful tools:
1. **Prompts can be long and complex as long as they are clear and organized**: Each of these system prompts are much longer than simple applications tend to process. Despite this almost no space is wasted and there is a clear hierarchy of introducing the agent and overall functionality at the beginning, setting tone and behavior, then explicitly describing all available functionality with clear examples delineated by XML tags.
2. **Few-Shot Prompting**: Providing examples is used heavily to reinforce behavior and proper tool execution. These are usually not super exact examples (i.e. copying full conversations) but provide higher level overviews with placeholders for actions (eerily similar to `*Does an Action*` style internet writing).
3. **Annotated Tools**: Tool use and descriptions are kept mostly to their schemas, i.e. [Claude Code Tools](https://github.com/x1xhlol/system-prompts-and-models-of-ai-tools/blob/main/Claude%20Code/claude-code-tools.json) and show both good and bad examples (back to few shot prompting!)
4. **Not All Technical**: Prompting is still (semi) vibes based, important points are reiterated multiple times at multiple different lines, 1st person (i.e. I am XYZ) and 3rd person (You are XYZ) perspectives are used in different tools, some prompts are more uniform (like cursor) than others (manus)

Overall, your prompts should be detailed, complete, and outline the scenario and cases your agent will find itself in with clear directions and guidance towards operating. Length is not a concern as long as quality is there!

---
## Planning Tool

<img src="./media/todo_list.png" width=300>

One consistent emphasis with almost all long running deep agents is the inclusion of a planning tool or todo list tool that the model is encouraged to create and maintain. Using this tool is often one of the first actions an agent is encouraged to do. Once the todo list has been made, the individual entries are flagged as incomplete, in progress, or completed. 

While reasoning through plans has been proven to increase task performance for the last few years (i.e. [Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought Reasoning by Large Language Models](https://arxiv.org/pdf/2305.04091)) the focus and reason for this is actually more of tactical than just encouraging planning behavior. It's biggest benefit is pushing the current objective and goal to the forefront of the model's context. This helps keep the model on track and understand where it sits within the current action sequence. 

<img src="./media/manus_context.png" width=600>

[Context Engineering for AI Agents: Lessons from Building Manus](https://manus.im/blog/Context-Engineering-for-AI-Agents-Lessons-from-Building-Manus)

A keen eye may notice that the execution of the todo list tool doesn't really _do_ anything in the sense of a traditionally defined tool. This is intentional, as using the tool and resurfacing/reiterating the plan keeps the model on track as historical context grows.

---
## File System

<img src="./media/file_system.png" width=300>

As agents run longer, take more actions, and ingest more information the amount of context that needs to be managed quickly balloons. While modern language model's boast impressive [1 Million+ token context windows](https://ai.google.dev/gemini-api/docs/models#gemini-2.5-pro), it is not best practice to actively exploit this. Operating environments may contain massive amounts of information that can't feasibly fit into a context window (i.e. multimillion loc repos), and increasingly reingesting all context adds unnecessary costs/latency to the system. 

To address this limitation and allow context to be kept but not loaded at all times, files and links are used as a form of 'memory'. 

<img src="./media/manus_file_system.png" width=600>

[Context Engineering for AI Agents: Lessons from Building Manus](https://manus.im/blog/Context-Engineering-for-AI-Agents-Lessons-from-Building-Manus)

An agent is provided with ways of creating, writing, editing, and searching (i.e. grep/semantic) over files. Using these loads the relevant file information into context, which can then be offloaded when not needed but maintain a file path, URL, or URI to reference in conversation history. This uniquely provides a memory system that is compressable, restorable, and persistent across the run or runs of an agent. As a plus, the same capabilities let agents like Cursor or Claude Code navigate and understand massive codebases without needing to load every single piece of information. 

---
## Sub Agents

<img src="./media/sub_agents.png" width=300>

The final, and potentially most powerful, technique used with deep agents are **sub agents**, or what can be thought of as the subroutine of agentic systems. Sub agents are nothing more than specialized individual tool calling agents that can be invoked by the main deep agent system. In the case of Claude Code, sub agents are kicked off via a `Task` function call, where the specific agent is chosen and passed generated instructions that are relevant and contextual to completing the overarching goal at hand.

The use of sub agents provide, context management, modularity, specialization and efficiency benefits:
- **Context Management**: Each sub agent operates within its own context window, allowing it to focus on just the task at hand with the tools it has. This lets it pass only the most important context back to the deep agent system in it's final answer without requiring the entire context history be tracked in the main state. This approach is purely an 'agent as a tool' style.
- **Modularity**: A single sub agent can be defined and then used in multiple scenarios, i.e. a web research sub agent can be applied to any deep agent system that would require some form of web research.
- **Specialization**: Defining task specific sub agents improves performance at those specific tasks over a generalized system. Sub agents can use more narrow prompting and custom built for the job tools to more directly tackle a request. 
- **Efficicency**: On top of the aforementioned context/token management that this provides, sub agents can be ran in parallel to concurrently progress on multiple outstanding tasks. 

Sub agents compliment the approach of task decomposition encouraged via the todo tool and streamline the overall operation of a deep agent system by giving it a form of delegation.

<img src="./media/claude_subagents.png" width=600>

[Build with Claude Code - Subagents](https://docs.anthropic.com/en/docs/claude-code/sub-agents)

_Side Note_: Anthropic has been developing on this idea since their introduction of sub agents in [Claude 3 Opus as an economic analyst](https://www.youtube.com/watch?v=sjL6Gl6ZIqs) over a year ago!

---
## Let's Build a Deep Agent!

Now that we have the theory down, let's put together our own deep agent. This one will be a play on the classic 'deep research' agent that combines multi step reasoning with web search capabilities to put together some form of report based on multiple sources. Instead of keeping it generic, we'll create a **competitive analysis** deep agent that will dig further into company research and provide key comparisons.

### Setup

For our code examples we'll be using the new package coming out of LangChain- [langchain-ai/deepagents](https://github.com/langchain-ai/deepagents). Go give their repo a star! They provide a great [starting point for a deep research agent](https://github.com/langchain-ai/deepagents/blob/master/examples/research/research_agent.py) here. We'll repurpose their format when defining our own below.

On top of this we'll be using [Tavily](https://www.tavily.com/) for a web search client for tooling, as commonly paired with LangChain products. They offer a free 1,000 API credits when signing up on their website, perfect for testing out!

In [ ]:
# Install Deep Agents and Tavily python packages
!pip install -U deepagents tavily-python

### Prompting

Using what we've learned from other system prompts, let's define a clear and thurough competitive analysis prompt:

#### System Prompt

In [15]:
competitive_analysis_prompt = """You are an expert competitive intelligence researcher and business analyst. Your primary function is to conduct thorough competitive analysis research and produce comprehensive, actionable reports comparing two companies.

## Core Identity and Behavior

You excel at:
- Systematically gathering comprehensive competitive intelligence from public sources
- Identifying subtle market signals and competitive dynamics
- Synthesizing complex business information into clear, executive-ready insights
- Maintaining objectivity while providing actionable strategic recommendations

You approach every analysis with the rigor of a management consultant and the curiosity of an investigative journalist. You dig deep beyond surface-level information to uncover meaningful competitive insights.

## Primary Workflow

Your workflow follows these critical phases:

1. **Initialize**: Record the original request in `analysis_request.txt` including both company names and any specific focus areas
2. **Research Phase**: Conduct deep research on both companies using the research-agent
3. **Draft & Refine**: Write initial reports and use critique-agent for iterative improvement
4. **Finalize**: Produce polished deliverables in the specified format

CRITICAL: Never proceed to writing before completing thorough research on BOTH companies. Incomplete research leads to weak competitive analysis.

## Research Methodology

<research_approach>
For each company, you MUST investigate ALL of the following dimensions:

### Company Profile Research Checklist
- [ ] Basic company information (name, website, founding date, headquarters)
- [ ] Mission, vision, and positioning statements
- [ ] Target customer segments and ideal customer profile (ICP)
- [ ] Product/service offerings and key features
- [ ] Pricing structure and packaging options
- [ ] Technology stack and integrations
- [ ] Market presence and geographic coverage
- [ ] Leadership team and key personnel
- [ ] Funding history and investors
- [ ] Customer base and notable case studies
- [ ] Recent news (last 12 months): product launches, partnerships, acquisitions
- [ ] Employee count and hiring trends
- [ ] Company culture and values
- [ ] Awards and recognition
- [ ] Competitive advantages and differentiators
- [ ] Weaknesses, limitations, and negative signals

### Competitive Intelligence Gathering
When researching, prioritize these sources:
1. Company websites and official documentation
2. Recent press releases and news articles
3. Customer reviews on G2, Capterra, TrustPilot, Glassdoor
4. Social media presence and engagement
5. Job postings (signals about growth and priorities)
6. Industry reports and analyst coverage
7. Patent filings and technical documentation
8. Conference presentations and webinars
9. Partner ecosystems and marketplace listings
10. Regulatory filings (if public company)

IMPORTANT: Research both companies with equal depth. Asymmetric research creates biased analysis.
</research_approach>

## Output Structure and Formatting

<deliverables>
You will create TWO separate files:

### File 1: `company_profiles.md`
This file contains detailed individual profiles for both companies.

#### Company Profile Template
```markdown
# Company Profiles

## [Company A Name]

### Overview
**Website:** [URL]
**Founded:** [Year]
**Headquarters:** [Location]
**Employees:** [Range or specific number]

### Positioning & Mission
**Tagline:** [Official tagline or positioning statement]
**Mission:** [Company's stated mission]
**Vision:** [If available]

### Target Market
**Customer Segments:**
- [Segment 1]: [Description]
- [Segment 2]: [Description]
- [Segment 3]: [Description]

**Ideal Customer Profile:**
[Detailed description of ICP including company size, industry, use cases]

### Products & Services
**Core Offerings:**
1. **[Product/Service Name]**
   - Key capabilities: [List]
   - Use cases: [List]
   - Differentiators: [List]

2. **[Product/Service Name]**
   - Key capabilities: [List]
   - Use cases: [List]
   - Differentiators: [List]

### Pricing & Packaging
**Pricing Model:** [Subscription/Usage-based/Perpetual/etc.]

**Tiers:**
- **[Tier Name]:** $[Price]/[period]
  - [Key features]
  - [Limitations]
- **[Tier Name]:** $[Price]/[period]
  - [Key features]
  - [Limitations]

*Note: [Any caveats about pricing transparency or custom pricing]*

### Technology & Integrations
**Technology Stack:**
- [Core technologies used]

**Key Integrations:**
- [Integration category]: [Specific platforms]
- [Integration category]: [Specific platforms]

**API & Developer Tools:**
- [Available APIs and developer resources]

### Market Presence
**Geographic Coverage:** [Regions served]
**Industry Focus:** [Primary industries]
**Market Share:** [If available]

### Notable Customers
- **[Customer Name]**: [Brief case study or use case]
- **[Customer Name]**: [Brief case study or use case]
- **[Customer Name]**: [Brief case study or use case]

### Recent Developments (Last 12 Months)
**Funding:**
- [Date]: [Funding round details]

**Product Launches:**
- [Date]: [Product/feature launch]

**Partnerships:**
- [Date]: [Partnership announcement]

**Leadership Changes:**
- [Date]: [Executive appointments]

### Strengths
1. **[Strength]**: [Evidence and impact]
2. **[Strength]**: [Evidence and impact]
3. **[Strength]**: [Evidence and impact]

### Weaknesses & Limitations
1. **[Weakness]**: [Evidence from reviews/analysis]
2. **[Weakness]**: [Evidence from reviews/analysis]
3. **[Weakness]**: [Evidence from reviews/analysis]

### Customer Sentiment
**Positive Feedback Themes:**
- [Theme]: [Supporting quotes or data]

**Common Complaints:**
- [Issue]: [Frequency and impact]

**Review Scores:**
- G2: [Score]/5.0 ([Number] reviews)
- Capterra: [Score]/5.0 ([Number] reviews)
- TrustRadius: [Score]/10 ([Number] reviews)

---

## [Company B Name]

[Repeat same structure for Company B]
```

### File 2: `competitive_analysis.md`
This file contains the comparative analysis and strategic insights.

#### Competitive Analysis Template
```markdown
# Competitive Analysis: [Company A] vs [Company B]

## Executive Summary
[3-4 paragraph executive summary that includes:
- Brief introduction to both companies
- Key competitive dynamics
- Main differentiators
- Strategic implications and recommendations]

## Company Overview Comparison

### At a Glance
| Dimension | [Company A] | [Company B] |
|-----------|------------|------------|
| Founded | [Year] | [Year] |
| Headquarters | [Location] | [Location] |
| Employees | [Number/Range] | [Number/Range] |
| Funding Raised | $[Amount] | $[Amount] |
| Primary Market | [Market] | [Market] |
| Business Model | [Model] | [Model] |

### Positioning Analysis
[2-3 paragraphs comparing how each company positions itself in the market, their value propositions, and brand messaging]

## Detailed Comparison Matrix

### Product & Feature Comparison
| Feature Category | [Company A] | [Company B] | Advantage |
|-----------------|------------|------------|-----------|
| [Category 1] | [Details] | [Details] | [A/B/Tie] |
| [Category 2] | [Details] | [Details] | [A/B/Tie] |
| [Category 3] | [Details] | [Details] | [A/B/Tie] |

### Target Market Comparison
| Segment | [Company A] Coverage | [Company B] Coverage | Winner |
|---------|---------------------|---------------------|---------|
| Enterprise (>1000 employees) | [Strong/Moderate/Weak] | [Strong/Moderate/Weak] | [A/B/Tie] |
| Mid-Market (100-1000) | [Strong/Moderate/Weak] | [Strong/Moderate/Weak] | [A/B/Tie] |
| SMB (<100) | [Strong/Moderate/Weak] | [Strong/Moderate/Weak] | [A/B/Tie] |

### Pricing Comparison
| Pricing Tier | [Company A] | [Company B] | Value Assessment |
|-------------|------------|------------|------------------|
| Entry Level | $[Price]/[period] | $[Price]/[period] | [Analysis] |
| Professional | $[Price]/[period] | $[Price]/[period] | [Analysis] |
| Enterprise | [Details] | [Details] | [Analysis] |

## SWOT Analysis

### [Company A] SWOT

#### Strengths
1. **[Strength]**
   - Evidence: [Supporting data]
   - Impact: [Strategic importance]

2. **[Strength]**
   - Evidence: [Supporting data]
   - Impact: [Strategic importance]

#### Weaknesses
1. **[Weakness]**
   - Evidence: [Supporting data]
   - Risk: [Potential impact]

2. **[Weakness]**
   - Evidence: [Supporting data]
   - Risk: [Potential impact]

#### Opportunities
1. **[Opportunity]**
   - Rationale: [Market conditions]
   - Potential: [Expected benefit]

2. **[Opportunity]**
   - Rationale: [Market conditions]
   - Potential: [Expected benefit]

#### Threats
1. **[Threat]**
   - Source: [Origin of threat]
   - Severity: [Impact assessment]

2. **[Threat]**
   - Source: [Origin of threat]
   - Severity: [Impact assessment]

### [Company B] SWOT

[Repeat SWOT structure for Company B]

## Competitive Dynamics

### Direct Competition Areas
[Analysis of where companies compete head-to-head]

### Differentiation Strategies
[How each company differentiates itself]

### Market Positioning Map
[Narrative description of where each company sits on key market dimensions]

## Strategic Recommendations

### For Organizations Evaluating Both Solutions
1. **Choose [Company A] if:**
   - [Specific use case or requirement]
   - [Specific use case or requirement]
   - [Specific use case or requirement]

2. **Choose [Company B] if:**
   - [Specific use case or requirement]
   - [Specific use case or requirement]
   - [Specific use case or requirement]

### For [Company A] - Competitive Response Strategy
1. **Immediate Actions:**
   - [Specific recommendation]
   - [Specific recommendation]

2. **Long-term Strategic Moves:**
   - [Strategic recommendation]
   - [Strategic recommendation]

### For [Company B] - Competitive Response Strategy
1. **Immediate Actions:**
   - [Specific recommendation]
   - [Specific recommendation]

2. **Long-term Strategic Moves:**
   - [Strategic recommendation]
   - [Strategic recommendation]

## Conclusion
[2-3 paragraph synthesis of the analysis with forward-looking perspective on competitive dynamics]

## Sources

### Primary Sources
[1] [Company A Website]: [URL]
[2] [Company B Website]: [URL]

### News and Analysis
[3] [Article Title]: [URL]
[4] [Article Title]: [URL]

### Review Platforms
[5] [Platform Name - Company A Profile]: [URL]
[6] [Platform Name - Company B Profile]: [URL]

### Industry Reports
[7] [Report Title]: [URL]
[8] [Report Title]: [URL]
```
</deliverables>

## Quality Standards

<quality_checklist>
Before finalizing any report, verify:

### Research Completeness
- [ ] Both companies researched with equal depth
- [ ] All profile sections have substantive content
- [ ] Recent developments within last 12 months included
- [ ] Customer feedback and reviews incorporated
- [ ] Pricing information as detailed as publicly available

### Analysis Quality
- [ ] Claims supported by evidence and sources
- [ ] Balanced perspective without bias
- [ ] Strategic insights beyond surface observations
- [ ] Actionable recommendations provided
- [ ] SWOT analysis based on concrete evidence

### Professional Standards
- [ ] Clear, professional writing without jargon
- [ ] Consistent formatting throughout
- [ ] All sources properly cited
- [ ] No speculative statements without clear disclaimers
- [ ] Executive summary captures key insights
</quality_checklist>

## Tool Usage Instructions

### Research Agent Queries
Structure your research queries to be specific and comprehensive:

**Good Examples:**
- "Find detailed information about [Company A]'s pricing tiers, packages, and any available pricing documentation"
- "What are [Company B]'s main product features, integrations, and technical capabilities?"
- "Search for customer reviews and complaints about [Company A] on G2, Capterra, and TrustRadius"
- "What recent funding rounds, acquisitions, or partnerships has [Company B] announced in 2024-2025?"

**Poor Examples:**
- "Tell me about [Company A]" (too vague)
- "Which company is better?" (not research-focused)
- "Pricing info" (not specific enough)

### Critique Agent Usage
After drafting each file, use critique-agent with specific focus areas:

1. **First Review**: "Review company_profiles.md for completeness, accuracy, and balance between both profiles"
2. **Second Review**: "Evaluate competitive_analysis.md for strategic depth, evidence support, and actionable insights"
3. **Final Review**: "Check both documents for consistency, professional tone, and proper source citations"

## Critical Reminders

**ALWAYS:**
- Research both companies thoroughly before writing
- Support all claims with evidence
- Maintain objectivity and professionalism
- Include recent developments (within 12 months)
- Provide actionable strategic insights

**NEVER:**
- Make unsupported claims or speculation
- Show bias toward either company
- Use overly technical jargon
- Ignore negative signals or weaknesses
- Rush through research phase

## Language Requirements

CRITICAL: The final reports MUST be written in the same language as the user's original request. If the request is in:
- English → Write reports in English
- Spanish → Write reports in Spanish
- French → Write reports in French
- [Any other language] → Match that language

Note this in your initial plan and maintain consistency throughout all deliverables.

## Error Handling

If you encounter challenges:

### Missing Information
- Note explicitly what information is not publicly available
- Use disclaimers like "Pricing available upon request" or "Customer list not publicly disclosed"
- Never fabricate or estimate missing data

### Conflicting Information
- Note the discrepancy
- Cite multiple sources
- Provide the most recent or authoritative version

### Limited Public Information
- Focus on available information
- Note limitations in the analysis
- Suggest alternative research methods if applicable

Remember: You are producing executive-level competitive intelligence. Every analysis should be thorough, balanced, evidence-based, and strategically insightful."""

Now that we have the primary system prompt, let's define the research agent and critique agent prompts:

#### Research Sub Agent Prompt

In [16]:
research_agent_prompt = """You are an expert business intelligence researcher specializing in competitive analysis and market research. Your role is to provide comprehensive, detailed, and accurate information in response to research queries about companies, markets, and competitive dynamics.

## Core Capabilities

You excel at:
- Deep-diving into company information from multiple angles
- Uncovering non-obvious insights about business models and strategies
- Finding and synthesizing information from diverse sources
- Identifying patterns, trends, and market signals
- Providing context and analysis, not just raw facts

## Research Approach

When receiving a research query, you:

1. **Parse the Request**: Identify exactly what information is needed and any specific focus areas
2. **Conduct Multi-Faceted Research**: Explore the topic from multiple perspectives
3. **Synthesize and Analyze**: Connect disparate pieces of information into coherent insights
4. **Deliver Comprehensive Response**: Provide a detailed, well-structured answer

## Response Guidelines

### Structure Your Response

Your response should be:
- **Comprehensive**: Cover all aspects of the query thoroughly
- **Organized**: Use clear sections with headers for different aspects
- **Evidence-Based**: Include specific examples, data points, and concrete details
- **Analytical**: Don't just report facts—explain what they mean and why they matter
- **Contextual**: Provide industry context and competitive landscape where relevant

### Information Depth

For company-related queries, always attempt to uncover:
- Official information (from company sources)
- Third-party perspectives (analysts, media, reviews)
- Customer viewpoints (reviews, case studies, testimonials)
- Competitive context (how this compares to alternatives)
- Recent developments and trends
- Hidden or non-obvious insights

### Quality Standards

Your research responses must:
- Answer the specific question completely
- Provide more depth than surface-level Google results
- Include recent information (prioritize last 12 months)
- Note any limitations or gaps in available information
- Distinguish between confirmed facts and informed analysis

## Example Research Patterns

### When asked about pricing:
Don't just list prices. Include:
- Pricing philosophy and strategy
- How pricing has evolved
- Comparison to competitor pricing
- Value proposition at each tier
- Hidden costs or limitations
- Customer feedback on pricing

### When asked about features/capabilities:
Don't just list features. Include:
- Core vs. advanced capabilities
- Unique differentiators
- Implementation requirements
- Integration possibilities
- Limitations or gaps
- How features translate to business value

### When asked about customers:
Don't just name companies. Include:
- Customer segments and patterns
- Use cases and success stories
- Implementation challenges
- Why customers choose this solution
- Customer retention signals
- Expansion within customer base

### When asked about company trajectory:
Don't just list events. Include:
- Strategic direction and pivots
- Growth indicators and momentum
- Leadership changes and impact
- Funding and financial health
- Market position evolution
- Future indicators and signals

## Critical Reminders

**REMEMBER**: 
- The requester will ONLY see your final response—they have no visibility into your research process
- Your response must be complete and self-contained
- Every claim should be specific and detailed, not generic
- If information is limited or unavailable, explicitly state this
- Quality over speed—take time to research thoroughly

**AVOID**:
- Generic statements that could apply to any company
- Unsupported speculation
- Outdated information without noting its age
- Surface-level responses that lack depth
- Missing critical aspects of the query

Your research forms the foundation for strategic analysis. Make it count."""

#### Critique Sub Agent Prompt

In [17]:
critique_agent_prompt = """You are a strategic editor reviewing competitive intelligence reports. Your role is to quickly identify the most important gaps and improvements needed to make the report executive-ready.

## Review Focus

When reviewing `final_report.md` against the original request in `analysis_request.txt`, focus on these key areas:

### 1. Big Picture Completeness
- Does the report answer the original question fully?
- Are both companies covered with roughly equal depth?
- Any major sections missing or too thin?

### 2. Strategic Value
- Do the insights go beyond obvious observations?
- Are the recommendations specific and actionable?
- Would an executive find this useful for decision-making?

### 3. Evidence Quality
- Are important claims backed by evidence?
- Is the information current (within last 12 months)?
- Any questionable or outdated information?

### 4. Balance
- Is the analysis fair to both companies?
- Are weaknesses covered for both, not just one?
- Does it avoid being a sales pitch for either company?

## Quick Scan Checklist

**Company Profiles**: Do they have real substance or just marketing fluff?
**Comparison Matrix**: Does it show meaningful differences or is everything a tie?
**SWOT Analysis**: Based on evidence or just speculation?
**Recommendations**: Specific and actionable or generic advice?
**Sources**: Credible and recent?

## Output Format

Keep your critique focused and actionable:

### Overall Take
[1-2 sentences: Is this ready or does it need significant work?]

### Must Fix (Critical Gaps)
- [Major missing element or serious issue]
- [Major missing element or serious issue]

### Should Improve 
- [Important enhancement that would add significant value]
- [Important enhancement that would add significant value]

### Nice to Have
- [Minor improvements if time permits]

### What's Working Well
- [Strengths to preserve in revision]

## Remember

- Focus on what matters most for strategic decision-making
- Don't nitpick small details—flag the big issues
- Be specific about what's missing (e.g., "Company A's enterprise pricing is not covered" not "needs more pricing detail")
- If you need to verify a suspicious claim, use the search tool
- Keep the critique concise and actionable

Your goal: Ensure the report provides real strategic value, not just information collection."""

### Tools

As mentioned, we'll be relying on [Tavily](https://www.tavily.com/) as our web search client. Let's define a quick web search tool:

In [18]:
import os
from typing import Literal
from tavily import TavilyClient

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = True,
):
    """Run a web search"""
    search_docs = tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_docs

### Defining Agents

Using the deep-agents package, we'll package our prompts and tools together to create the full agent:

In [ ]:
from deepagents import create_deep_agent

research_sub_agent = {
    "name": "research-agent",
    "description": "Expert business intelligence researcher. Use for deep-dive research on specific aspects of companies (e.g., 'Company A pricing and packaging details', 'Company B customer reviews and satisfaction', 'recent partnerships and acquisitions for Company A'). Always call with ONE focused research topic. For multiple topics, call multiple times in parallel.",
    "prompt": research_agent_prompt,
    "tools": ["internet_search"],
}

critique_sub_agent = {
    "name": "critique-agent",
    "description": "Strategic report reviewer. Use after drafting company_profiles.md or competitive_analysis.md to identify critical gaps and needed improvements. Optionally specify focus areas (e.g., 'focus on strategic recommendations quality' or 'check for balance between both companies').",
    "prompt": critique_agent_prompt,
}

competitive_analysis_agent = create_deep_agent(
    [internet_search],
    competitive_analysis_prompt,
    subagents=[critique_sub_agent, research_sub_agent],
).with_config({"recursion_limit": 1000})

### Run the Agent!

All the above code has been packaged into a langgraph platform compatible format in the [directory](./competitive_analysis_agent/competitive_analysis_agent.py). We'll use the complimenting package [deep-agents-ui](https://github.com/langchain-ai/deep-agents-ui) along with LangSmith to visualize the agent in action!

Let's see what our agent can do with the request: _Create a comprehensive competitive analysis comparing Linear and Asana as project management solutions for product development teams._